# 1. Import and Install Dependencies

In [ ]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
import mediapipe as mp

# 2. MP Holistic and MP drawing for key points

In [ ]:
media_landmarks = mp.solutions.holistic # Holistic model
media_drawing = mp.solutions.drawing_utils # Drawing utilities
#Holistic model is designed to perform simultaneous detection of body language signals from a person’s face, hands, and pose.
#utilities provide functions helps to draw the landmarks onto the images

In [ ]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [ ]:
#function for drawing connections between landmark points of pose, face and both hands.
def draw_landmarks(image, results):
    media_drawing.draw_landmarks(image, results.face_landmarks, media_landmarks.FACEMESH_CONTOURS) # Draw face connections
    media_drawing.draw_landmarks(image, results.pose_landmarks, media_landmarks.POSE_CONNECTIONS) # Draw pose connections
    media_drawing.draw_landmarks(image, results.left_hand_landmarks, media_landmarks.HAND_CONNECTIONS) # Draw left hand connections
    media_drawing.draw_landmarks(image, results.right_hand_landmarks, media_landmarks.HAND_CONNECTIONS) # Draw right hand connections

In [ ]:
#function for drawing landmark points.
def draw_styled_landmarks(image, results):
    media_drawing.draw_landmarks(image, results.face_landmarks, media_landmarks.FACEMESH_CONTOURS, 
                             media_drawing.DrawingSpec(color=(255,244,79), thickness=1, circle_radius=1), 
                             media_drawing.DrawingSpec(color=(255,244,79), thickness=1, circle_radius=1)
                             ) 
    media_drawing.draw_landmarks(image, results.pose_landmarks, media_landmarks.POSE_CONNECTIONS,
                             media_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                             media_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    media_drawing.draw_landmarks(image, results.left_hand_landmarks, media_landmarks.HAND_CONNECTIONS, 
                             media_drawing.DrawingSpec(color=(255,100,100), thickness=2, circle_radius=4), 
                             media_drawing.DrawingSpec(color=(250,90,90), thickness=2, circle_radius=2)
                             ) 
    media_drawing.draw_landmarks(image, results.right_hand_landmarks, media_landmarks.HAND_CONNECTIONS, 
                             media_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             media_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

In [ ]:
cap = cv2.VideoCapture(0)
with media_landmarks.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()
        frame = cv2.flip(frame, 1)
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        draw_styled_landmarks(image, results)
        cv2.imshow('OpenCV Feed', image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

In [ ]:
plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

# 3. Extract Keypoint Values

In [ ]:
# extracting and flattening the pose, face, left hand, and right hand landmarks from the results. If no landmarks detected returns a zero array of a specific
# length . The lengths of the arrays (132 for pose, 1404 for face, and 63 for each hand) correspond to the number of
# landmarks multiplied by the number of coordinates (and visibility for pose) for each landmark
pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)
face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(1404)
lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)

In [ ]:
# Extracts and concatenates pose, face, left hand, and right hand landmarks from the results.
# If no landmarks are detected for a category, a zero array of a specific length is returned.
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

In [ ]:
result_test = extract_keypoints(results)

In [ ]:
result_test.shape

# 4. Setup Folders for Collection

In [ ]:
#creating folder for data collection
DATA_PATH = os.path.join('MP_Data')
actions = np.array(['Hello', 'Namaste', 'Bye', 'Thanks', 'Indian', 'I', 'man', 'woman'])
no_sequences = 30
sequence_length = 10

In [ ]:
for action in actions: 
    for sequence in range(1,no_sequences+1):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

# 5. Collect Keypoint Values for Training and Testing

In [ ]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with media_landmarks.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:

    for action in actions:
        # Loop through sequences or videos
        for sequence in range(1,no_sequences+1):
            # Loop through video length or sequence length
            for frame_num in range(sequence_length):

                ret, frame = cap.read()
                # Make detections
                image, results = mediapipe_detection(frame, holistic)
                # Draw landmarks
                draw_styled_landmarks(image, results)

                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(1000)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                
                # NEW Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                    
    cap.release()
    cv2.destroyAllWindows()

In [ ]:
#in case if want to delete test data
# import shutil
# dir = os.path.join(DATA_PATH)
# shutil.rmtree(dir)

# 6. Preprocess Data and Create Labels and Features

In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [ ]:
label_map = {label:num for num, label in enumerate(actions)}

In [ ]:
label_map

In [ ]:
sequences, labels = [], []
for action in actions:
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [ ]:
X = np.array(sequences)

In [ ]:
X.shape

In [ ]:
y = to_categorical(labels).astype(int)

In [ ]:
y

In [ ]:
y.shape

In [ ]:
#split training testing data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [ ]:
X_train.shape

In [ ]:
y_train.shape

# 7. Build and Train LSTM Neural Network

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping

In [ ]:
log_dir = os.path.join('Logs1')
tb_callback = TensorBoard(log_dir=log_dir)

In [ ]:
#adding lstm layers for real time prediction
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(10,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(8, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
callback = EarlyStopping(monitor='loss', patience=10)
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=500, callbacks=[tb_callback])

In [ ]:
%tensorboard --logdir Logs1

# 8. Make Predictions

In [ ]:
res = model.predict(X_test)

In [ ]:
for i in range(5):
    print(actions[np.argmax(res[i])])

In [ ]:
for i in range(5):
    print(actions[np.argmax(y_test[i])])

# 10. Evaluation using Confusion Matrix and Accuracy

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [ ]:
yhat = model.predict(X_train)

In [ ]:
ytrue = np.argmax(y_train, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [ ]:
multilabel_confusion_matrix(ytrue, yhat)

In [ ]:
accuracy_score(ytrue, yhat)

# 11. Test in Real Time

In [ ]:
from scipy import stats

In [ ]:
colors = []
for _ in range(len(label_map)):
    num = np.random.randint(0,256)
    colors.append((num,num,num))

In [ ]:
colors

In [ ]:
colors = []
for _ in range(len(label_map)):
    num = np.random.randint(0,256)
    colors.append((num,num,255))

def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [ ]:
plt.figure(figsize=(18,18))
# plt.imshow(prob_viz(res, actions, image, colors))

In [ ]:
sequence = []
sentence = []
threshold = 0.5

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with media_landmarks.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()
        image, results = mediapipe_detection(frame, holistic)
        draw_styled_landmarks(image, results)
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-10:]
        if len(sequence) == 10:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            
            if res[np.argmax(res)] > threshold: 
                if len(sentence) > 0: 
                    if actions[np.argmax(res)] != sentence[-1]:
                        sentence.append(actions[np.argmax(res)])
                else:
                    sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        cv2.imshow('OpenCV Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()